# User interaction analysis (based on direct interactions)

This notebook includes analysis on direct user interactions, focusing on:
 - Identification of user clusters within r/antiwork using spectral clustering

A direct interaction is defined as an interaction between a parent post/comment and a child comment)

In [15]:
from scipy.sparse import load_npz
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import polars as pl
from pathlib import Path
import Clustering
import importlib as imp

path = "../../data/users/"
adj_matrix_path = path + 'adj_matrix-directs-latest.npz'
DB_PATH = path + 'users.sqlite.db'

In [16]:
#load the adjacency matrix
adj_matrix = load_npz(adj_matrix_path).tolil()
adj_matrix.setdiag(0) #set diagonals to zero to remove any "self-interactions"
A = adj_matrix.toarray()

In [17]:
#load users
conn_string = "sqlite://" + str(Path(DB_PATH).absolute())
selected_users = pl.read_sql("SELECT * FROM users WHERE is_selected ORDER BY matrix_id ASC", conn_string)

## Clustering

We run spectral clustering and explore how well it performs with up to 30 clusters. 

In [18]:
selected_vecs, results, sizes, eigvalues = Clustering.spectral_clustering(A, max_clusters=30)

Computing eigenvalues..
Running K-means..


In [19]:
Clustering.plot_cluster_diagnostics(results, sizes, eigvalues)

alt.HConcatChart(...)

Based on the above outputs, we can see that there aren't really any strong clusters. In cases where silhouette scores are high, most users simply belong to a single cluster.

### Clustering based on normalized user interactions

What if instead the same analysis was normalized to total user activity? We adjust the adjacency matrix by normalizing for total activity that each user has among in the top user network, which makes the edges represent the weight of interactions among users rather than the absolute interactions themselves.

In [20]:
norm_A = np.nan_to_num(A / np.sum(A, axis=1), 0) * 100

/tmp/ipykernel_3364/2628922934.py:1: RuntimeWarning: invalid value encountered in true_divide
  norm_A = np.nan_to_num(A / np.sum(A, axis=1), 0) * 100


In [21]:
selected_vecs, results, sizes, eigvalues = Clustering.spectral_clustering(norm_A, max_clusters=30)

Computing eigenvalues..
Running K-means..


In [22]:
Clustering.plot_cluster_diagnostics(results, sizes, eigvalues)

alt.HConcatChart(...)

As the silhouette scores are a bit higher, let's choose one of the configurations with the highest score (clusters == 11) and analyze what clusters have been generated.

In [24]:
optimal_clusters = 11
vecs = selected_vecs[:,:optimal_clusters]
cluster_model = MiniBatchKMeans(n_clusters=optimal_clusters, max_iter=1000, random_state=42).fit(vecs)
selected_users['cluster'] = cluster_model.labels_
imp.reload(Clustering)
Clustering.get_basic_stats(selected_users, cut_off = 10).to_pandas().round(1)

,Statistic,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7
0,cluster,8.0,0.0,7.0,6.0,5.0,4.0,3.0,9.0
1,Total number of users,1462.0,42.0,22.0,110.0,87.0,21.0,17.0,6035.0
2,Median posts per user,0.5,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,Median comments per user,135.0,140.0,138.0,183.0,137.0,138.0,126.0,162.0
4,Median activity window (days),124.0,196.0,118.0,139.5,157.0,143.0,331.0,125.0
5,Median average post karma,0.0,0.0,0.0,6.8,8.0,0.0,0.0,4.0
6,Median average comment karma,10.7,5.5,6.6,9.3,11.9,8.7,6.0,11.6


The results are quite similar. We find some very small clusters of users (< 60 users) which are likely arising due to interacting on a single post thread.